# Express




### Basic Server




#### setup code

How to use nodejs express?

express service?

express website?


In [ ]:
const express = require('express')
const cors = require('cors')
const router = express.Router();
const cookieParser = require('cookie-parser');
const importService = importer.import('import new service')
const WebSocket = require('ws');
const crypto = require('crypto')

process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

const TIMEOUT = 30 * 1000
let ROOT = process.env.DEFAULT_ROOT
let DEFAULT = process.env.DEFAULT_ROOT
let SERVICES = [
  'node express.ipynb',
  'documentation.ipynb',
]

let server

function defaultHandler(root, name, req, res, next) {
  if(!req.cookies.session_id) {
    res.cookie('session_id', (req.cookies.session_id = crypto.randomBytes(16).toString('hex')))
  }

  if (decodeURIComponent(req.path) == root) {
    return next(new Error(name + ' not loading'))
  }
  if (root) {
    console.info('redirecting:', req.path, root)
    return res.redirect(root)
  } else {
    return next()
  }
}

async function startServer(services) {
  if (!services) {
    services = SERVICES
  }
  if (typeof services == 'string') {
    services = services.split(/,\s*/gi)
  }

  var app = express()

  const server = http.createServer(app);

  server.keepAliveTimeout = TIMEOUT;
  server.headersTimeout = TIMEOUT;
  // Create a WebSocket server instance, passing the HTTP server
  const wss = new WebSocket.Server({ server });

  wss.on('connection', (ws, req) => {
    const { subscribeSocket } = importer.import('websocket subscription service')
    console.log('Client connected');
    try {
      subscribeSocket(ws, req)
    } catch (e) {
      console.log(e)
    }
  });

  app.use(cors({
    credentials: true,
    origin: '*'
  }))
  app.use(cookieParser());
  app.use(express.json({ limit: '20mb' }))
  app.use(express.urlencoded({ extended: true }));

  app.get('/import/:query', importService.bind(null, app, router))

  let boundRoot = defaultHandler.bind(null, ROOT, 'Root')
  boundRoot.name = 'defaultHandler'
  router.all('/', boundRoot)
  boundDefault = defaultHandler.bind(null, DEFAULT, 'Default')
  boundDefault.name = 'defaultHandler'
  router.all('*', boundDefault)

  // call the service directly to boot
  await new Promise(resolve => importService(app, router, { params: { query: services } }, { redirect: resolve }))

  app.use(router)
  let port = parseInt(process.env.EXPRESS_PORT || '8080')
  server.listen(port, () => console.log('Listening on http://localhost:' + port))

}

function closeExpress(activity) {
  if (activity) {
    return
  }
  if (server) {
    server.close()
    server = null
  }
}

module.exports = {
  startServer,
  closeExpress,
}




### import new service?


In [ ]:
const {loadRoutes} = importer.import('express router')

async function importService(app, router, req, res, next) {
  let service = req.params['query']
  let codeCells = importer.interpret(service)
  let thisRoute
  let thisDefault
  let thisRoot
  let routeParams
  let rootParams
  let defaultParams
  if (typeof codeCells.markdown != 'undefined') {
    codeCells = [codeCells]
  }

  // remove our own default services
  const routes = router.stack;
  for(let i = routes.length - 1; i >= 0; i--) {
    if(routes[i].handle.name === 'defaultHandler'
      || routes[i].path == '/'
      || routes[i].path == '*'
      || routes[i].path == '/*'
      || (routes[i].path == void 0
        && routes[i].route
        && (routes[i].route.path == '/'
        || routes[i].route.path == '*'))
    ) {
      routes.splice(i, 1)
    }
  }

  for (let i = 0; i < codeCells.length; i++) {
    //let hasFunction = codeCells[i].code.match(/(export default|export async function|export function|__all__ =|module.exports =)/gi)
    // TODO: multi language support
    //if(!hasFunction) {
    //  continue
    //}
    let { route, root, defaultRoute, params } = loadRoutes(app, router, codeCells[i])
    // TODO: choose the last setup as the default?
    if (route) {
      thisRoute = route
      routeParams = params
    }
    if (root) {
      thisRoot = root
      rootParams = params
    }
    if (defaultRoute) {
      thisDefault = defaultRoute
      defaultParams = params
    }
  }

  // TODO: reassign default handlers
  if(thisRoot) {
    console.log('creating route:', '/', thisRoot, rootParams[rootParams.length-1].name)
    router.all('/', ...rootParams.slice(1))
  }
  if(thisDefault) {
    console.log('creating route:', '*', thisDefault, defaultParams[defaultParams.length-1].name)
    router.all('*', ...defaultParams.slice(1))
  }

  let route = thisRoot || thisDefault || thisRoute
  if(route === true) {
    console.log('error route not detected:' + service)
    return next(new Error('Route not detected'))
  }
  return res.redirect(route)
}

module.exports = importService




### websocket subscription service?


In [ ]:
const path = require('path')
const chokidar = require('chokidar')
const cookie = require('cookie');
const WATCHERS = {}
const WATCHES = {}
const TRACES = []
const EVENTS = {}
//const PINGS = []

function traceConsole(notebook, logger, name, ...args) {
  TRACES.forEach(ws => {
    ws.send(JSON.stringify({ log: name, notebook: notebook.map(notebook => path.basename(notebook)).filter((a, i, arr) => arr.indexOf(a) === i) }));
  });
}

function subscribeSocket(ws, req) {
  console.trace(traceConsole)
  const cookies = cookie.parse(req.headers.cookie || '');
  ws.session_id = cookies.session_id

  ws.on('message', async data => {
    let message = data
    try {
      message = JSON.parse(data)
    } catch (e) { }

    if (message != 'ping') {
      console.log('Received:', message);
    }
    if (message.type === 'events') {
      let watchFile = importer.interpret(message.notebook).filename
      if (typeof EVENTS[path.basename(watchFile)] === 'undefined') {
        EVENTS[path.basename(watchFile)] = []
      }
      EVENTS[path.basename(watchFile)].push(ws)
    }

    if (message.type === 'subscribe') {
      // TODO: push notifications of executed functions to client
      TRACES.push(ws)
      ws.send(JSON.stringify({ tracing: true }));
    }
    if (message.type === 'refresh') {
      let watchFile = importer.interpret(message.service).filename
      if (typeof WATCHES[watchFile] == 'undefined') {
        WATCHES[watchFile] = []
        // TODO: initialize chokidar
        WATCHERS[watchFile] = chokidar.watch(watchFile, {
          interval: 1000,
          atomic: 1000,
          awaitWriteFinish: true
        });
        WATCHERS[watchFile].on("change", function (filepath, event) {
          WATCHES[watchFile].forEach(ws => {
            ws.send(JSON.stringify({ refresh: true }));
          });
        })
      }
      WATCHES[watchFile].push(ws)
      ws.send(JSON.stringify({ subscribed: true }));
    }
    if (message.type === 'rpc') {
      const { rpcHandler } = importer.import('express rpc service')
      let result
      await rpcHandler({
        body: message,
        query: message,
        params: message,
        cookies: message,
        headers: {
          accept: 'application/json'
        }
      }, {
        send: (data) => result = JSON.stringify(data),
        json: (json) => result = JSON.stringify(json),
        sendFile: (data) => result = JSON.stringify(data), // TODO: ?
      }, (error) => result = JSON.stringify(error))
      ws.send(result);
    }
  });

  ws.on('close', () => {
    // TODO: remove ourselves from subscriptions
    console.log('Client disconnected');
  });
}

module.exports = {
  subscribeSocket,
  TRACES,
  EVENTS
}


### express rpc service

express rpc service?

ROUTE[] = /rpc

ROUTE[] = /rpc/:function

METHOD = all


In [ ]:
const crypto = require('crypto')

function overrideSends(service, req, obj, original, ...args) {
  let socketService = importer.interpret('realiable-websocket.js').code
    + '\n' + importer.interpret('browser socket service').code
  if (args[0] && (args[0] + '').match(/<\/body>/gi) && !(args[0] + '').match(/\/\/ browser socket service/g)) {
    args[0] = (args[0] + '').replaceAll(/<\/body>/gi, `
      <script>// browser socket service\n` + socketService.replaceAll('{SERVICE}', service) + `\n</script></body>`)
    args[0] = (args[0] + '').replaceAll(/<\/head>/gi, `<script>
      const SUBSCRIPTIONS = ["${service}"];
      const TRACES = [];
      const EVENTS = [];
    </script>
    </head>`)

  } else if (args[0] && (args[0] + '').match(/<\/body>/gi)) {
    args[0] = (args[0] + '').replaceAll(/<\/body>/gi, `
      <script>// browser socket service\nSUBSCRIPTIONS.push('${service}')\n</script></body>`)
  }

  if(!req.cookies.session_id) {
    res.cookie('session_id', (req.cookies.session_id = crypto.randomBytes(16).toString('hex')))
  }

  return original.apply(obj, [...args])
}

async function serviceHandler(service, req, res, next) {
  const getParameters = await importer.import('function parameters')
  if (!service) {
    return next(new Error('No service'))
  }
  let then = Date.now()
  console.log('importing:', service)

  let codeCell = await importer.interpret(service)
  if (!codeCell) {
    return next(new Error('No code cell'))
  }
  if (codeCell.language == 'css') {
    res.setHeader('content-type', 'text/css');
    return res.send(codeCell.code)
  }
  console.log(codeCell.markdown, codeCell.markdown.match(/ROUTE.*\.js(\n|$)/gi))
  if (codeCell.language == 'javascript' && codeCell.markdown.match(/ROUTE.*\.js(\n|$)/gi)) {
    res.setHeader('content-type', 'text/javascript');
    return res.send(codeCell.code)
  }
  if (codeCell.language == 'html') {
    return overrideSends(service, req, res, res.send, codeCell.code)
  }
  let serviceFunction
  try {
    serviceFunction = await importer.import(service)
  } catch (e) {
    console.log(e)
    return next(e)
  }
  if (typeof serviceFunction == 'object' && typeof serviceFunction[Object.keys(serviceFunction)[0]] == 'function') {
    serviceFunction = serviceFunction[Object.keys(serviceFunction)[0]]
  }
  // TODO: support multiple languages parameters more nicely than importer.run()
  let isRPC = true
  let parameters

  if (codeCell.language == 'javascript') {
    parameters = getParameters(codeCell.code).slice(1)
    console.log('parameters:', parameters)
    if ((parameters.includes('res') || parameters.includes('response'))
      && (parameters.includes('req') || parameters.includes('request'))) {
      isRPC = false
    } else {
      isRPC = true
    }
  }

  // TODO python

  console.log('importing took:', ((Date.now() - then) / 1000) + 's')
  // TODO C#

  try {
    if (isRPC && parameters) {
      let inputs = []
      for (let i = 0; i < parameters.length; i++) {
        inputs.push(req.body[parameters[i]] || req.params[parameters[i]] || req.query[parameters[i]] || req.cookies[parameters[i]])
      }
      then = Date.now()
      console.info('calling:', codeCell.id, inputs)
      let result = await serviceFunction.apply(null, inputs)
      console.info('calling took:', ((Date.now() - then) / 1000) + 's')
      if (typeof result == 'object' || req.headers.accept == 'application/json' || codeCell.markdown.match(/ROUTE.*\.json(\n|$)/gi))
        return res.json(result)
      else
        return overrideSends(service, req, res, res.send, result)
    } else {
      //if (!res.isOverriden) {
      let originalSend = res.send
      then = Date.now()
      res.send = overrideSends.bind(null, service, req, res, originalSend)
      console.info('calling:', codeCell.id)
      let result = await serviceFunction(req, res, next)
      console.info('calling took:', ((Date.now() - then) / 1000) + 's')
      return result
      //} else {
      //  return await serviceFunction(req, res, next)
      //}
    }
  } catch (e) {
    console.log(e)
    return next(e)
  }
}

async function rpcHandler(req, res, next) {
  return await serviceHandler(req.body['function'] || req.query['function'] || req.params['function'], req, res, next)
}

module.exports = {
  rpcHandler,
  serviceHandler
}



### browser socket service?

ROUTE = /socket-service.js


In [ ]:


//const socket = new WebSocket(window.location.origin);

const socket = new ReconnectableWebSocket(window.location.origin, {
  maxRetries: Infinity,
  heartbeatInterval: 1000
});

socket.addEventListener('open', () => {
  console.log('Connected to server');
  SUBSCRIPTIONS.forEach(service => {
    socket.send(JSON.stringify({ type: 'refresh', service: service }));
  })
  if (TRACES && TRACES.length > 0) {
    socket.send(JSON.stringify({ type: 'subscribe' }));
  }
  EVENTS.forEach(({ notebook }) => {
    socket.send(JSON.stringify({ type: 'events', notebook: notebook }));
  })

});

socket.addEventListener('message', (event) => {
  let message = event.data
  try {
    message = JSON.parse(event.data)
  } catch (e) { }
  console.log('Message from server:', message);
  if (message.refresh === true) {
    window.location = window.location.origin + window.location.pathname + window.location.search
  }
  if (message.event) {
    EVENTS.forEach(({ callback, notify }) => {
      if(message.event === notify) {
        callback(message)
      }
    })
  }
  if (message.log) {
    TRACES.forEach(callback => callback(message))
  }
});


socket.addEventListener('close', () => console.log('Disconnected'));
socket.addEventListener('error', (err) => console.error('WebSocket error:', err));



reliable-websocket.js?

ROUTE = /reliable-websocket.js


In [ ]:
class ReconnectableWebSocket extends EventTarget {
  constructor(url, options = {}) {
    super();
    this.url = url;
    this.autoReconnect = true;
    this.maxRetries = options.maxRetries || Infinity;
    this.heartbeatInterval = options.heartbeatInterval || 30000;
    this.backoffBase = options.backoffBase || 1000;
    this.backoffMax = options.backoffMax || 30000;
    this.retryCount = 0;
    this.ws = null;
    this.heartbeatTimer = null;
    this.connect();
  }

  connect() {
    this.ws = new WebSocket(this.url);

    this.ws.addEventListener('open', (event) => {
      console.log('[WebSocket] Connected');
      this.retryCount = 0;
      this.startHeartbeat();
      this.dispatchEvent(new Event('open'));
    });

    this.ws.addEventListener('message', (event) => {
      this.dispatchEvent(new MessageEvent('message', { data: event.data }));
    });

    this.ws.addEventListener('close', (event) => {
      console.warn('[WebSocket] Closed', event.code);
      this.stopHeartbeat();
      this.dispatchEvent(new CloseEvent('close', event));
      this.reconnect();
    });

    this.ws.addEventListener('error', (event) => {
      console.error('[WebSocket] Error', event);
      this.dispatchEvent(new Event('error'));
    });
  }

  reconnect() {
    if (!this.autoReconnect || this.retryCount >= this.maxRetries) return;
    const delay = Math.min(
      this.backoffBase * 2 ** this.retryCount + Math.random() * 1000,
      this.backoffMax
    );
    console.log(`[WebSocket] Reconnecting in ${(delay / 1000).toFixed(1)}s...`);

    setTimeout(() => {
      this.retryCount++;
      this.connect();
    }, delay);
  }

  startHeartbeat() {
    this.stopHeartbeat();
    this.heartbeatTimer = setInterval(() => {
      if (this.ws.readyState === WebSocket.OPEN) {
        this.ws.send('ping');
      }
    }, this.heartbeatInterval);
  }

  stopHeartbeat() {
    clearInterval(this.heartbeatTimer);
  }

  send(data) {
    if (this.ws.readyState === WebSocket.OPEN) {
      this.ws.send(data);
    } else {
      console.warn('[WebSocket] Cannot send, not open.');
    }
  }

  close() {
    this.autoReconnect = false;
    this.stopHeartbeat();
    this.ws.close();
  }
}


### express auto router

express router?


In [ ]:
const {serviceHandler} = importer.import('express rpc service')
const parseEnvironment = importer.import('parse environment')
const express = require('express')

process.env['NODE_TLS_REJECT_UNAUTHORIZED'] = 0;

const LOADED = {}

function loadRoutes(app, router, codeCell) {
  let env = parseEnvironment(codeCell.markdown)

  let routes = env['ROUTE']
  if (typeof routes == 'string') {
    routes = [env['ROUTE']]
  }
  //let hasFunction = codeCell.code.match(/(export default|export async function|export function|__all__ =|module.exports =)/gi)
  // TODO: multi language support
  if (!routes && codeCell.questions[0]) {
    routes = ['/rpc/' + codeCell.questions[0]]
    if (env['ROOT'] && env['ROOT'] == 'true') {
      return { root: routes[0] }
    }
    if (env['DEFAULT'] && env['DEFAULT'] == 'true') {
      return { defaultRoute: routes[0] }
    }
    return { route: routes[0] }
  }
  if (!routes) {
    return {}
  }

  let firstParams
  for (let i = 0; i < routes.length; i++) {
    // TODO: load dynamically by finding the root in router and removing it and readding
    if (LOADED[codeCell.id + routes[i]]) {
      continue
    }
    LOADED[codeCell.id + routes[i]] = true

    console.log('creating route:', routes[i], (env['STATIC'] ? env['STATIC'] : codeCell.questions[0]))

    let params = [routes[i]]
    let auth
    if (env['AUTHENTICATE'] && (auth = eval(env['AUTHENTICATE']))) {
      let authenticate = serviceHandler.bind(null, auth)
      params.push(authenticate)
    }
    if (env['STATIC']) {
      if (typeof env['STATIC'] == 'string') {
        params.push(express.static(eval(env['STATIC'])))
      } else {
        params.push(express.static(eval(env['STATIC'][i])))
      }
    } else {
      let boundHandler = serviceHandler.bind(null, codeCell.questions[0])
      boundHandler.name = codeCell.questions[0]
      params.push(boundHandler)
    }
    if (env['STATIC']) {
      app.use(...params)
    } else if (env['METHOD'] == 'all') {
      router.all(...params)
    } else if (env['METHOD'] == 'post') {
      app.post(...params)
    } else {
      app.get(...params)
    }
    if(!firstParams) {
      firstParams = params
    }
  }

  let result = {
    route: routes[0],
    params: firstParams
  }
  if (env['ROOT'] && env['ROOT'] == 'true') {
    result.root = routes[0]
  }
  if (env['DEFAULT'] && env['DEFAULT'] == 'true') {
    result.defaultRoute = routes[0]
  }
  return result

}

module.exports = {
  loadRoutes,
  LOADED,
}



parse environment?


In [ ]:

function parseEnvironment(markdown) {

  const SCAN_ENVIRONMENT = /([A-Z_\[\]]*)\s*[\:-=]+\s*(.*?)\s*(\n|$)/g

  let env = {}
  while ((match = SCAN_ENVIRONMENT.exec(markdown)) !== null) {
    if (match[1].endsWith('[]')) {
      let key = match[1].substring(0, match[1].length - 2)
      if (typeof env[key] == 'undefined') {
        env[key] = []
      }
      env[key].push(match[2])
    } else {
      env[match[1]] = match[2]
    }
  }

  return env
}

module.exports = parseEnvironment



#### initialize

How to start a proxy using express?

How to start a proxy on {port}?



In [ ]:
if (typeof server != 'undefined') {
    server.close();
}
var app = express();
app.use('/', function (req, res) {
    var url = host + req.url;
    req.pipe(request(url)).pipe(res);
});

$$.async();
try {
    var server = app.listen(port, () => $$.done('server up and running on port ' + port));
    server.on('error', (e) => $$.done(e));
} catch (e) {
    $$.done(e);
}




#### send user a link to itself

How to display the output from express?

How to test express is working with HTML?

In [ ]:
$$.mime({
    'text/html': `
<iframe id="sosmethod" name="sosmethod" 
    style="height:600px; width:100%; border:none;" 
    src="http://${host}:${port}/"></iframe>`
});


#### dom query using JSDOM

How to run a DOM query on a remote HTML page?

In [ ]:
$$.async();
var images = [];
new Promise((resolve, reject) => {
    request('http://' + host + ':' + port, (err, res, body) => {
        if (err) {
            reject(err)
        }
        else {
            resolve(body)
        }
    });
}).then((body) => {
    var nodeList = (new JSDOM(body)).window.document.querySelectorAll(query);
    return Array.prototype.slice.call(nodeList, 0)
        .map(i => i.getAttribute('src'));
}).then(sources => {
    images = sources;
    $$.done(sources);
}).catch(e => $$.done(e));


#### extract images

How to save a list of image elements?

How to display a list of image elements in markdown?

How to {output} image elements in the requested {format}?


In [ ]:
var html = '';
var htmlPrint = '';
var requests = images.map((src) => {
    var filename = src.split('/').pop();
    return new Promise((resolve, reject) => {
        request(src, {encoding: 'binary'}, (err, res, body) => {
            if (err) {
                reject(err)
            }
            else {
                resolve(body)
            }
        });
    }).then(body => {
        return new Promise((resolve, reject) => {
            fs.writeFile(
                path.join(output, filename),
                body,
                'binary',
                function (err) {
                    if (err) reject(err);
                    else resolve({filename: filename, data: body});
                });
        })
    });
});
$$.async();
Promise.all(requests).then(images => {
    images.forEach((i) => {
        var filename = i.filename.split('/').pop();
        var ext = mime.lookup(i.filename);
        html += '<img src="data:' + ext + ';base64,' + (new Buffer(i.data, 'binary')).toString('base64') + '" />';
        htmlPrint += '<li><img src="/assets/' + filename + '" /></li>\n';
    });
    $$.mime({'text/markdown': 'Usage:\n\n```html\n' + htmlPrint + '\n```\nOutput:\n' + html});
}).catch(e => $$.done(e));


#### stop server

How to stop express server?

In [ ]:
if (typeof server !== 'undefined') {
    server.close();
}




## RPC Wrapper



#### the code

express rpc wrapper

legacy rpc wrapper


In [ ]:
var importer = require('../Core');
var { getResult } = importer.import(['get rpc permissions', 'rpc result']);

async function rpcHandler(req, res) {
    console.log('rpc function:', req.body['function'] || req.query['function'] || req.params['function'])
    let functionName = req.body['function'] || req.query['function'] || req.params['function']
    if(!functionName) {
        return res.status(500).send(new Error('It works!'));
    }
    try {
        let r = getResult({
            command: functionName,
            result: importer.interpret(functionName),
            body: req.method === 'POST' ? req.body : req.query,
            circles: ['Public']
        })
        res.status(200).send(JSON.stringify(r, null, 4));
    } catch (e) {
        const resultError = Object.getOwnPropertyNames(e).reduce((alt, key) => {
            alt[key] = e[key];
            return alt;
        }, {});
        console.log(e);
        res.status(500).send(JSON.stringify(resultError, null, 4))
    }
}

module.exports = rpcHandler



#### test mock

old BS I wrote for a job that wasn't appreciated.

zuora eloqua express mock?


In [ ]:
var bodyParser = require('body-parser');
var express = require('express');

var selenium = express();
var server = require('http').createServer(selenium);
selenium.use(bodyParser.json());    // to support JSON-encoded bodies
selenium.use(bodyParser.urlencoded({// to support URL-encoded bodies
    extended: true
}));

var router = express.Router();


// auth endpoints
router.post('/auth/oauth2/authorize', (req, res) => { 
    res.send('');
});

router.post('/auth/oauth2/token', (req, res) => {
    res.send({
        "access_token": "access_token",
        "token_type": "bearer",
        "expires_in": 28800,
        "refresh_token": "refresh_token",
    })
});


// eloqua endpoints
router.post('/bulk/2.0/contacts/imports', (req, res) => {
    res.send({
        uri: '/imports/1234'
    });
});
router.post('/bulk/2.0/imports/:importId/data', (req, res) => {
    res.status(204);
    res.send({
        uri: '/imports/1234'
    });
});
router.post('/bulk/2.0/syncs', (req, res) => {
    res.send({
        uri: '/sync/1234'
    });
});
var alternateImportStatus = true;
router.get('/bulk/2.0/sync/:syncId', (req, res) => {
    res.send({
        status: alternateImportStatus ? 'active' : 'success',
    });
    alternateImportStatus = !alternateImportStatus;
});


// eloqua custom data object
// as opposed to /bulk/2.0/customobjects/imports which just lists all the imports that have been performed on the CDO
router.post('/bulk/2.0/customobjects/:objectId/imports', (req, res) => {
    res.send({
        uri: '/imports/1234'
        // might be? /customobjects/:objectId/imports/:importId
    });
});




// zuora endpoints
router.post('/object/export', (req, res) => {
    res.send({
        Id: '1234'
    });
});
var alternateExportStatus = true;
router.get('/object/export/:exportId', (req, res) => {
    res.send({
        Status: alternateExportStatus ? 'Processing' : 'Completed',
        FileId: '1234'
    });
    alternateExportStatus = !alternateExportStatus;
});
router.get('/files/:fileId', (req, res) => {
    res.send(`RatePlanCharge.Description,Account.Id
support,123456
premium,654321
`);
});

selenium.use(router);
var app;
function restart() {
    if(typeof app !== 'undefined') {
        app.close();
    }
    app = server.listen(18888).on('error', e => {
        if(e.code !== 'EADDRINUSE') {
            throw e;
        }
    });
    return app;
}
restart();
app.restart = restart;
module.exports = app;

process.on ('SIGTERM', () => {app.close(); process.exit(0)});
process.on ('SIGINT', () => {app.close(); process.exit(0)});



## Tools



### Directory to HTML

directory to html?


In [ ]:
const fs = require('fs');
const path = require('path');

function directoryToHtml(dirPath, back, session, bodyOnly) {
  let files = fs.readdirSync(dirPath)

  files.sort()
  files.sort((a, b) => {
    let aIsDir = fs.statSync(path.join(dirPath, a)).isDirectory()
    let bIsDir = fs.statSync(path.join(dirPath, b)).isDirectory()

    if (aIsDir && !bIsDir) return -1; // a is dir, b is file
    if (!aIsDir && bIsDir) return 1; // b is dir, a is file

    return a.localeCompare(b);
  })
  let htmlOutput = `<html>
<head>
<style>
ul {
  list-style: none;
  padding: 0;
  margin: 0;
}

li {
  padding: 10px;
  border-bottom: 1px solid #ccc;
}

li:last-child {
  border-bottom: none;
}

i {
  font-size: 18px;
  margin-right: 10px;
  color: #666;
}

span {
  font-size: 16px;
  color: #333;
}

ul {
  padding-left: 20px;
}

ul li {
  padding-left: 10px;
}
</style>
</head>  
<body><h1>Directory Listing</h1><ul>`

  if(bodyOnly) 
    htmlOutput = `<ul>`;

  if(back) {
    htmlOutput += `<li><a href="../?t=${Date.now()}${session ? ('&session=' + session): ''}">Parent directory/</a></li>`;
  }

  files.forEach((file) => {
    const filePath = path.join(dirPath, file);
    const fileStat = fs.statSync(filePath);

    if(file[0] == '.' || file.includes('porn') || file.includes('nsfw') || file.includes('naked') || file.includes('x-rated')) {
      return
    }

    if (fileStat.isDirectory()) {
      htmlOutput += `<li><a href="${encodeURIComponent(file).replaceAll('(', '%28').replaceAll(')', '%29')}/?t=${Date.now()}${session ? ('&session=' + session): ''}">${file}/</a></li>`;
    } else {
      htmlOutput += `<li><a href="${encodeURIComponent(file).replaceAll('(', '%28').replaceAll(')', '%29')}?t=${Date.now()}${session ? ('&session=' + session): ''}">${file}</a></li>`;
    }
  });

  if(bodyOnly) 
    htmlOutput += '</ul>';
  else
    htmlOutput += '</ul></body></html>';

  return htmlOutput
}

module.exports = directoryToHtml



### Directory Listing

node express directory handler?

DEFAULT = true

AUTHENTICATE = process.env.DEFAULT_AUTHENTICATOR

ROUTE[] = /home

ROUTE[] = /home/*


In [ ]:
const path = require('path')
const fs = require('fs')
const BASE_DIRECTORY = process.env.BASE_DIRECTORY || process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE
const directoryToHtml = importer.import('directory to html')

async function handleDirectory(req, res, next) {
  let basePath = path.resolve(BASE_DIRECTORY)
  let segments = req.path.split('/')
  let routeSegments = req.route.path.split('/')
  componentPath = routeSegments.map((segment, i) => segment
    .replaceAll(/:([^\/]*)/gi, (_, $1) => req.params[$1])
    .replaceAll(/\*/gi, () => i == routeSegments.length - 1 ? '' : segments[i])
  ).join('/')
  let relativePath = path.relative(componentPath, decodeURIComponent(req.path))
  console.log('relative directory:', componentPath, relativePath)
  let combined = path.join(basePath, relativePath)
  console.log('http directory:', combined)

  if(!fs.existsSync(combined)) {
    console.error('Path not found:', combined)
    return next(new Error('Path not found: ' + combined))
  }

  if(path.extname(combined).toLocaleLowerCase() == '.mp3') {

    if(req.url.includes('?raw=true')) {
      return res.sendFile(combined)
    }

    return res.send(importer.interpret('html music player').code
      .replaceAll('{FILE_URL}', req.path)
      .replaceAll('{SESSION}', req.authenticated ? req.authenticated : ''))
  }

  if((req.authenticated || combined.includes('stable-diffusion-webui/outputs')) 
    && (path.extname(combined).toLocaleLowerCase() == '.png' 
    || path.extname(combined).toLocaleLowerCase() == '.jpg' 
    || path.extname(combined).toLocaleLowerCase() == '.jpeg')
  ) {

    if(req.url.includes('?raw=true')) {
      return res.sendFile(combined)
    }

    return res.send(importer.interpret('html image viewer').code
      .replaceAll('{FILE_URL}', req.path)
      .replaceAll('{SESSION}', req.authenticated ? req.authenticated : ''))
  }

  if(req.authenticated && fs.statSync(combined).isFile()) {
    return res.sendFile(combined)
  }

  if(!fs.statSync(combined).isDirectory()) {
    return res.status(403).send('Forbidden')
  }

  let html = await directoryToHtml(combined, relativePath.split('/').length > 2, req.authenticated)
    .replaceAll('<head>', '<head><base href="http://localhost:8080' + req.path + (req.path.endsWith('/') ? '' : '/') + '" />')

  return res.send(html)
}

module.exports = {
  handleDirectory,
  BASE_DIRECTORY
}



### Music Player

html music player?


In [ ]:
<head>
<style>
ul {
  list-style: none;
  padding: 0;
  margin: 0;
}

li {
  padding: 10px;
  border-bottom: 1px solid #ccc;
}

li:last-child {
  border-bottom: none;
}

i {
  font-size: 18px;
  margin-right: 10px;
  color: #666;
}

span {
  font-size: 16px;
  color: #333;
}

ul {
  padding-left: 20px;
}

ul li {
  padding-left: 10px;
}
</style>
<link rel="stylesheet" type="text/css" href="/amplitude.css" />
<script src="/amplitude.js"></script>
</head>  
<body><h1>Directory Listing</h1><ul>
<li><a href="./?session={SESSION}">Parent directory/</a></li>
</ul>
<div id="visualizations-player">
  <div class="top-container">
    <img class="now-playing-album-art" id="large-now-playing-album-art" data-amplitude-song-info="cover_art_url"/>
    <div class="amplitude-visualization" id="large-visualization">

    </div>
    <div class="visualization-toggle visualization-on"></div>
    <div class="amplitude-shuffle"></div>
    <div class="amplitude-repeat"></div>
  </div>

  <div class="meta-data-container">
    <span class="now-playing-name" data-amplitude-song-info="name"></span>
    <span class="now-playing-artist-album">
      <span class="now-playing-artist" data-amplitude-song-info="artist"></span> - <span class="now-playing-album" data-amplitude-song-info="album"></span>
    </span>
  </div>

  <div class="amplitude-wave-form">

  </div>
  <input type="range" class="amplitude-song-slider" id="global-large-song-slider"/>

  <div>
    <span class="amplitude-current-time"></span>

    <span class="amplitude-time-remaining"></span>
  </div>

  <div class="control-container">
    <div class="amplitude-prev">

    </div>

    <div class="amplitude-play-pause amplitude-paused">

    </div>

    <div class="amplitude-next">

    </div>
  </div>

  <div class="song-navigation">
    <input type="range" class="amplitude-song-slider"/>
  </div>

  <div class="arrow-up">
    <img src="https://521dimensions.com/img/open-source/amplitudejs/examples/visualizations/arrow-up.svg" class="arrow-up-icon"/>
  </div>

  <div id="visualizations-player-playlist">
    <div class="top-arrow">
      <img src="https://521dimensions.com/img/open-source/amplitudejs/examples/visualizations/arrow-down.svg" class="arrow-down-icon"/>
    </div>

    <div class="top">
      <span class="playlist-title">Songs</span>
      <div class="amplitude-repeat">

      </div>
      <div class="amplitude-shuffle">

      </div>
    </div>

    <div class="songs-container">
      <div class="song amplitude-song-container amplitude-play-pause"  data-amplitude-song-index="0">
        <span class="song-position">01</span>
        <img class="song-album-art" data-amplitude-song-info="cover_art_url" data-amplitude-song-index="0"/>
        <div class="song-meta-data-container">
          <span class="song-name" data-amplitude-song-info="name" data-amplitude-song-index="0"></span>
          <span class="song-artist" data-amplitude-song-info="artist" data-amplitude-song-index="0"></span>
        </div>
      </div>
    </div>

    <div class="active-audio">
      <img class="cover-art-small" data-amplitude-song-info="cover_art_url"/>

      <div class="active-audio-right">
        <span class="song-name" data-amplitude-song-info="name"></span>

        <div class="active-audio-controls">
          <div class="amplitude-prev"></div>
          <div class="amplitude-play-pause"></div>
          <div class="amplitude-next"></div>
        </div>
      </div>
    </div>
  </div>
</div>
<script>
  Amplitude.init({
		songs: [
			{
				"name": "Song Name 1",
				"artist": "Artist Name",
				"album": "Album Name",
				"url": "{FILE_URL}?raw=true" + ('{SESSION}'.length > 0 ? ('&session={SESSION}') : ''),
				"cover_art_url": "/cover/art/url.jpg"
			},
    ]
  })
</script>
</body></html>




### Image View

html image viewer?


In [ ]:
<head>
  <style>
    ul {
      list-style: none;
      padding: 0;
      margin: 0;
    }

    li {
      padding: 10px;
      border-bottom: 1px solid #ccc;
    }

    li:last-child {
      border-bottom: none;
    }

    i {
      font-size: 18px;
      margin-right: 10px;
      color: #666;
    }

    span {
      font-size: 16px;
      color: #333;
    }

    ul {
      padding-left: 20px;
    }

    ul li {
      padding-left: 10px;
    }
  </style>
  <link rel="stylesheet" type="text/css" href="/amplitude.css" />
  <script src="/amplitude.js"></script>
</head>

<body>
  <h1>Directory Listing</h1>
  <ul>
    <li><a href="./?session={SESSION}">Parent directory/</a></li>
  </ul>
  <img src="{FILE_URL}?raw=true&session={SESSION}" />
</body>

</html>


### rpc index?

show a full rpc index list of services and callout functionality.

ROUTE = /list

ROOT = true


In [ ]:
const path = require('path')
var { listInProject } = importer.import('list project files');
const { functionCache } = importer.import('cache rpc functions with llm descriptions')
const { LOADED } = importer.import('express router')
const parseEnvironment = importer.import('parse environment')


function rpcList(req, res, next) {
  let wrapperFragment = importer.interpret('rpc service index').code
  let repoFragment = importer.interpret('rpc fragment').code
  let repositoriesFragment = importer.interpret('rpc list fragment').code
  let repositoriesHtml = ''
  let then = Date.now()

  const notebooks = listInProject(path.resolve(__dirname, '../'), '{,*,*/*,*/*/*,*/*/*/*}*.ipynb')
  console.log('parsing all notebooks:', then)
  for (let i = 0; i < notebooks.length; i++) {
    let cells
    try {
      cells = importer.interpret(path.basename(notebooks[i]))
    } catch (up) {
      if (up.message.includes('Nothing found')) {
        continue
      } else {
        throw up
      }
    }
    for (let j = 0; j < cells.length; j++) {
      //const hasFunction = cells[j].code.match(/(export default|export async function|export function|__all__ =|module.exports =)/gi)
      const cache = functionCache[notebooks[i] + '[' + j + ']']
      const env = parseEnvironment(cells[j].markdown)
      if (cells[j].questions[0] || env['ROUTE']) {
        let route = typeof LOADED[cells[j].id] == 'undefined'
          ? ('/import/' + path.basename(cells[j].filename))
          : (env['ROUTE'] ? (typeof env['ROUTE'] == 'string' ? env['ROUTE'] : env['ROUTE'][0]) : ('/rpc/' + cells[j].questions[0]))

        let fragment = repoFragment
          .replaceAll('{TITLE}', cells[j].questions[0] || (typeof env['ROUTE'] == 'string' ? env['ROUTE'] : env['ROUTE'][0]))
          .replaceAll('{ROUTE}', route)
          .replaceAll('{DESCRIPTION}', cache ? cache.summary
            .replaceAll('&', '&amp;')
            .replaceAll('$', '&dollar;')
            .replaceAll('<', '&lt;')
            .replaceAll('>', '&gt;') : cells[j].markdown)

        repositoriesHtml += fragment
      }
    }
  }
  console.log('parsing all notebooks took:', ((Date.now() - then) / 1000) + 's')

  res.send(wrapperFragment
    .replaceAll('{USERNAME}', '' /* path.basename(PROFILE_HOME)*/)
    .replaceAll('{TIMESTAMP}', Date.now())
    .replaceAll('{MAINCONTENT}', repositoriesFragment
      .replaceAll('{REPOSITORIES}', repositoriesHtml)
    )
    //.replaceAll('&dollar;', '$$')
  )
}


module.exports = rpcList



rpc list fragement?


In [ ]:
<section class="repositories">
  <h2>RPC functions</h2>
  <div class="repo-list">
      {REPOSITORIES}
  </div>
</section>



rpc fragement?


In [ ]:
<div class="repo-item">
  <h3><a href="{ROUTE}">{TITLE}</a></h3>
  <p>{DESCRIPTION}</p>
</div>



rpc service index?


In [ ]:
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Forgotten RPC</title>
    <link rel="stylesheet" href="/express-styles.css?t={TIMESTAMP}">
</head>

<body>
    <!-- Header Section -->
    <header>
        <nav class="navbar">
            <div class="logo">
                <a href="#">Forgotten RPC</a>
            </div>
            <div class="search">
                <input type="text" placeholder="Search APIs" id="search">
            </div>
            <!--<div class="user-profile">
                <a href="#">User Profile</a>
            </div>-->
        </nav>
    </header>

    <!-- Main Content Section -->
    <div class="container">
        <!-- Sidebar Section -->
        <aside class="sidebar">
            <ul>
                <li><a href="/list">Functions</a></li>
                <li><a href="/swagger">Swagger Spec</a></li>
            </ul>
        </aside>

        <!-- Main Content Area -->
        <main class="main-content">
            {MAINCONTENT}
        </main>
    </div>

    <!-- Footer Section -->
    <footer>
        <p>&copy; 2025 Git Lost. All rights reserved.</p>
    </footer>
    <script>
        document.addEventListener("keydown", function (event) {
            if (event.key === "." && !event.ctrlKey && !event.metaKey) {
                event.preventDefault();
                window.location.href = window.location.href.replace("github.com", "github.dev");
            }
        })
    </script>
</body>

</html>



node express styles?

ROUTE = /express-styles.css


In [ ]:
/* General Reset */
* {
    margin: 0;
    padding: 0;
    box-sizing: border-box;
}

/* Body Styles */
body {
    font-family: Arial, sans-serif;
    background-color: #333;
    color: #FFF;
}

/* Header Styles */
header {
    background-color: #24292f;
    color: #fff;
    padding: 10px 20px;
}

body > header {
    position: fixed;
    top: 0;
    left: 0;
    right: 0;
    z-index: 1000;
}

h1, h2, h3, h4, h5 {
    color: #fff;
    padding: 0 0 20px 0;
}

.navbar {
    display: flex;
    justify-content: space-between;
    align-items: center;
}

.navbar .logo a {
    font-size: 24px;
    color: #fff;
    text-decoration: none;
    font-weight: bold;
}

.navbar .search input {
    padding: 8px 15px;
    font-size: 14px;
    border-radius: 4px;
    border: 1px solid #ddd;
    width: 300px;
}

.navbar .user-profile a {
    color: #fff;
    text-decoration: none;
    font-size: 14px;
}

/* Main Layout */
.container {
    display: flex;
    margin-top: 80px;
    margin-bottom: 80px;
    padding: 0 20px;
}

/* Sidebar Styles */
.sidebar {
    width: 220px;
    background-color: #24292f;
    color: #fff;
    padding-top: 20px;
    margin-right: 20px;
}

.sidebar ul {
    list-style-type: none;
}

.sidebar ul li {
    padding: 10px 10px;
}

.sidebar ul li a {
    color: #fff;
    text-decoration: none;
    font-size: 16px;
    display: block;
    padding: 5px 10px;
}

.repositories h2 {
  font-size: 24px;
  margin-bottom: 20px;
}

.repo-list {
  display: flex;
  flex-direction: column;
  gap: 15px;
}

.repo-item {
  color: #f9fafb;
  padding: 15px;
  border-radius: 6px;
  box-shadow: 0 1px 3px rgba(0, 0, 0, 0.1);
}

.repo-item a {
  color: #f9fafb;
  text-decoration: none;
}

.repo-item a:hover {
  text-decoration: underline;
}

.repo-item:hover {
  background-color: #3c4043;
}
